# Collecting and cleaning the puzzle data
---
This notebook contains details on the collection and processing of the following datasets.

- Lichess puzzle databse.
- My puzzle activity.
- My rating history.

The **Lichess puzzle database** was obtained from https://database.lichess.org/#puzzles on March 22, 2023. The database contains information on Lichess puzzles including

- Puzzle ID
- [FEN](https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation) (board position 1 move prior to the beginning of the puzzle)
- Moves (i.e. solution to the puzzle in [UCI format](https://en.wikipedia.org/wiki/Universal_Chess_Interface))
- Rating
- Rating Deviation
- Popularity
- Number of Plays
- Themes
- Game URL
- Opening Tags.

From the [documentation](https://database.lichess.org/#puzzles):

>Generating these chess puzzles took more than 50 years of CPU time.
We went through 300,000,000 analysed games from the Lichess database, and re-analyzed interesting positions with Stockfish 12/13/14/15 NNUE at 40 meganodes. The resulting puzzles were then automatically tagged. To determine the rating, each attempt to solve is considered as a Glicko2 rated game between the player and the puzzle. Finally, player votes refine the tags and define popularity.

We'll also investigate two different, but related, datasets regarding my puzzle performance. First is my **puzzle activity**, which is a `.ndjson` file tracking every puzzle I have ever played on [Lichess](https://lichess.org), including date, time, win/loss, and the puzzle's rating. The other dataset collects my **rating history**, which is a `.json` file with information on my puzzle rating by day.

# Import the required packages
---

We'll use the following packages.
- [`requests`](https://pypi.org/project/requests/) to access the [Lichess API](https://lichess.org/api).
- [`pandas`](https://pandas.pydata.org/docs/) for data processing.
- [`json`](https://docs.python.org/3/library/json.html) for reading the data.
- [`datetime`](https://docs.python.org/3/library/datetime.html) for working with dates in the raw data.


In [ ]:
import requests
import pandas as pd
import json
import datetime

# Lichess puzzle database
---

## Collection

The [Lichess puzzle database](https://database.lichess.org/#puzzles) is originally in a compressed `.csv.zst` format. While `read_csv` from `pandas` is supposed to be able to read such files, I first had to decompress the database to a `.csv` file in the command line. For more detail as to how this is done, see the `Zstandard` [repository on GitHub](https://github.com/facebook/zstd).

Once the file is decompressed, we use `read_csv` to create a dataframe named `puzzles_df`, then preview the first 5 rows as well as the last 5 rows.

In [ ]:
puzzles_df = pd.read_csv('lichess_db_puzzle.csv', header = None)

In [ ]:
puzzles_df.head()

,0,1,2,3,4,5,6,7,8,9
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1925,74,95,3136,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1518,75,96,19577,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN
2,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1172,75,86,538,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
3,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1565,81,89,80,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN
4,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1520,75,100,169,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN


In [ ]:
puzzles_df.tail()

,0,1,2,3,4,5,6,7,8,9
3157075,zzzYp,3r1rk1/3pN1pp/nq3pn1/1pp1p1QN/4P3/5P2/2P3PP/3R...,g8h8 e7g6 h7g6 g5g6,1058,77,89,590,crushing kingsideAttack middlegame short,https://lichess.org/Yc8Fdtc0/black#52,NaN
3157076,zzzc4,3r3k/p5pp/8/5R2/1BQ1p3/P3q3/Bb4PP/6K1 w - - 0 28,g1f1 d8d1 b4e1 e3e1,1022,75,89,364,mate mateIn2 middlegame short,https://lichess.org/jRTmvh6y#55,NaN
3157077,zzzco,5Q2/pp3R1P/1kpp4/4p3/2P1P3/3PP2P/Pr2q3/2K5 w -...,f7f2 b2c2 c1b1 e2d1,1711,75,88,1076,endgame mate mateIn2 queensideAttack short,https://lichess.org/hZWTYIAT#69,NaN
3157078,zzzhI,r3kb1r/ppp2ppp/2n5/3q3b/3P1B2/5N1P/PPP3P1/RN1Q...,c6d4 f1e1 e8d8 b1c3 d4f3 g2f3,2492,88,82,62,advantage long opening,https://lichess.org/N092KH4f/black#20,Kings_Gambit_Accepted Kings_Gambit_Accepted_Ab...
3157079,zzzhg,r2q1rk1/4bppp/p1n1pn2/1p1pN3/2pP2b1/1PP1P3/PBQ...,g4f5 e5c6 f5c2 c6d8 c2b1 d8c6,2252,74,94,129,crushing hangingPiece long middlegame,https://lichess.org/f4w3Ljxp/black#24,Queens_Pawn_Game Queens_Pawn_Game_Colle_System


Note, there are over 3 million puzzles in the database!

## Processing

### Add headers

The original data has no headers—from https://database.lichess.org/#puzzles, the column information is
- Puzzle ID
- FEN
- Moves
- Rating
- Rating Deviation
- Popularity
- Number of Plays
- Themes
- Game URL
- Opening Tags.

Let's rename the columns in our dataframe accordingly and preview the result.

In [ ]:
puzzles_df.rename({0: 'Puzzle_ID', 
                1: 'FEN', 
                2: 'Moves', 
                3: 'Rating', 
                4: 'Rating_Deviation', 
                5: 'Popularity', 
                6: 'Number_of_Plays', 
                7: 'Themes', 
                8: 'URL', 
                9: 'Opening_Tags'}, 
               axis='columns', 
               inplace=True)

puzzles_df.head()

,Puzzle_ID,FEN,Moves,Rating,Rating_Deviation,Popularity,Number_of_Plays,Themes,URL,Opening_Tags
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1925,74,95,3136,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1518,75,96,19577,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN
2,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1172,75,86,538,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
3,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1565,81,89,80,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN
4,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1520,75,100,169,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN


### Missing or null values

Let's check how many missing or null values each column has.

In [ ]:
puzzles_df.isna().sum(axis = 0)

Puzzle_ID                 0
FEN                       0
Moves                     0
Rating                    0
Rating_Deviation          0
Popularity                0
Number_of_Plays           0
Themes                    0
URL                       0
Opening_Tags        2433504
dtype: int64

There are quite a few missing values in the `Opening_Tags` column. According to the [documentation](https://database.lichess.org/#puzzles),

>The OpeningTags field is only set for puzzles starting before move 20.

There are generally considered to be 3 stages of a chess game, namely
- the [opening](https://en.wikipedia.org/wiki/Chess_opening),
- the [middlegame](https://en.wikipedia.org/wiki/Chess_middlegame), and
- the [endgame](https://en.wikipedia.org/wiki/Chess_endgame).

While there are no explicit, quantitative distinctions between these stages, a tactic occuring within the first 20 moves of a game likely has features strongly influenced by the opening played, whereas puzzles occuring later may not be as strongly influenced by the opening. So, it makes sense that only puzzles occuring within the first 20 moves of the game have opening tags.

### Add a column for puzzle length
I'd like to add a column called `Puzzle_Length` that counts the number of moves needed to complete each puzzle. The `Moves` column is a space-separated string of moves in [UCI format](https://en.wikipedia.org/wiki/Universal_Chess_Interface). As noted in the documentation,

>The position to present to the player is after applying the first move...

So, the length of the puzzle is 1 less than the number of moves listed in the `Moves` column.



In [ ]:
Puzzle_Length = puzzles_df['Moves'].map(lambda x: len(x.split()) - 1)
puzzles_df['Puzzle_Length'] = Puzzle_Length

puzzles_df.head()

,Puzzle_ID,FEN,Moves,Rating,Rating_Deviation,Popularity,Number_of_Plays,Themes,URL,Opening_Tags,Puzzle_Length
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1925,74,95,3136,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN,5
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1518,75,96,19577,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN,3
2,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1172,75,86,538,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...,3
3,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1565,81,89,80,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN,5
4,000Zo,4r3/1k6/pp3r2/1b2P2p/3R1p2/P1R2P2/1P4PP/6K1 w ...,e5f6 e8e1 g1f2 e1f1,1520,75,100,169,endgame mate mateIn2 short,https://lichess.org/n8Ff742v#69,NaN,3


### Save it for later

We save the processed data for future use.

In [ ]:
puzzles_df.to_csv('lichess_db_puzzle_clean.csv', index = False)

# My puzzle activity
---

## Collection

The puzzle activity data is accessible from the Lichess API at https://lichess.org/api/puzzle/activity. The request needs an authorization—I used a personal token generated from https://lichess.org/account/oauth/token (anonymized in the code below). For more information on the Lichess API, see https://lichess.org/api.

In [ ]:
url = 'https://lichess.org/api/puzzle/activity'
token = 'xxxx'
headers = {'Authorization' : 'Bearer ' + token}

We now make the request with the authorization headers from above (it may take awhile to run).

In [ ]:
puzzle_activity = requests.get(url, headers = headers)

Check the status code to make sure the request was successful—an output of `200` will indicate a successful request.


In [ ]:
puzzle_activity.status_code

200

## Processing

### Reading the data

According to the [Lichess API Reference](https://lichess.org/api), the response is a `.ndjson` file (i.e. a new-line delimited `.json`—so, each line is a `.json` object). This caused some issues when trying to create a dataframe, so we'll split the `.ndjson` file into a list of `.json` objects that `.json_normalize` has an easier time understanding.

Below, you can see the response as a plain text string. Each line represents a puzzle that I played, indicating 
- the puzzle's ID (`id`),
- date and time the puzzle was played (in a 13-digit, millisecond-precise format),
- whether I successfully completed the puzzle (`win` being `true` or `false`), and
- the puzzle's rating (`puzzleRating`).

In [ ]:
puzzle_activity.text

'{"id":"nhQA7","date":1680207258719,"win":true,"puzzleRating":1877}\n{"id":"0BoYj","date":1680207158469,"win":true,"puzzleRating":1643}\n{"id":"Yq8Ld","date":1679939422328,"win":true,"puzzleRating":1962}\n{"id":"49u5p","date":1679939380924,"win":true,"puzzleRating":1917}\n{"id":"mQPeh","date":1679939344291,"win":true,"puzzleRating":1798}\n{"id":"ki6uI","date":1679939263623,"win":true,"puzzleRating":1814}\n{"id":"eK4RQ","date":1679939167887,"win":true,"puzzleRating":1705}\n{"id":"vvgFs","date":1679697472084,"win":true,"puzzleRating":1836}\n{"id":"tKORm","date":1679697333115,"win":true,"puzzleRating":1788}\n{"id":"lJOYT","date":1679697119924,"win":true,"puzzleRating":1693}\n{"id":"Aiv94","date":1679660925402,"win":true,"puzzleRating":1875}\n{"id":"lL4KQ","date":1679658569209,"win":true,"puzzleRating":1881}\n{"id":"NiKqi","date":1679610812197,"win":true,"puzzleRating":1702}\n{"id":"iPbFA","date":1679437917076,"win":true,"puzzleRating":1661}\n{"id":"qLTL1","date":1679437837696,"win":true,"

Note that each puzzle entry of `puzzle_activity.text` is delimited by `\n`. We split this string into a list and preview the first 5 entries. 

In [ ]:
puzzle_activity_list = puzzle_activity.text.split('\n')

puzzle_activity_list[0:5]

['{"id":"nhQA7","date":1680207258719,"win":true,"puzzleRating":1877}',
 '{"id":"0BoYj","date":1680207158469,"win":true,"puzzleRating":1643}',
 '{"id":"Yq8Ld","date":1679939422328,"win":true,"puzzleRating":1962}',
 '{"id":"49u5p","date":1679939380924,"win":true,"puzzleRating":1917}',
 '{"id":"mQPeh","date":1679939344291,"win":true,"puzzleRating":1798}']

The elements of `puzzle_activity_list` are `string` objects, not `json` objects, though. So, we use `json.loads` to parse each `string` as a `json` object and compile the results into a list of `json` objects called `puzzle_activity_dicts`.

In [ ]:
puzzle_activity_dicts = []

for i in range(len(puzzle_activity_list) - 1):
  puzzle_activity_dicts.append(json.loads(puzzle_activity_list[i]))
    
puzzle_activity_dicts[0:5]

[{'id': 'nhQA7', 'date': 1680207258719, 'win': True, 'puzzleRating': 1877},
 {'id': '0BoYj', 'date': 1680207158469, 'win': True, 'puzzleRating': 1643},
 {'id': 'Yq8Ld', 'date': 1679939422328, 'win': True, 'puzzleRating': 1962},
 {'id': '49u5p', 'date': 1679939380924, 'win': True, 'puzzleRating': 1917},
 {'id': 'mQPeh', 'date': 1679939344291, 'win': True, 'puzzleRating': 1798}]

Now, `puzzle_activity_dicts` is a list of `json` objects, with each entry representing a puzzle that I played. We can convert this to a dataframe using `json_normalize` which can read each entry of `puzzle_activity_dicts` as a `json` object and enter it as a row in the dataframe.

In [ ]:
puzzle_activity_df = pd.json_normalize(puzzle_activity_dicts)

puzzle_activity_df.head()

,id,date,win,puzzleRating
0,nhQA7,1680207258719,True,1877
1,0BoYj,1680207158469,True,1643
2,Yq8Ld,1679939422328,True,1962
3,49u5p,1679939380924,True,1917
4,mQPeh,1679939344291,True,1798


### Missing or null values

Let's check for missing or null values.

In [ ]:
puzzle_activity_df.isna().sum(axis = 0)

id              0
date            0
win             0
puzzleRating    0
dtype: int64

There are no missing or null values!

### Converting the dates

The `date` column is in 13-digit format, which is a millisecond-precise timestamp. To make this easier to interpret, we'll divide these timestamps by `1000` and convert them to a datetime.

In [ ]:
new_dates = puzzle_activity_df['date'].map(lambda x: datetime.datetime.fromtimestamp(x / 1000))
puzzle_activity_df['date'] = new_dates

puzzle_activity_df.head()

,id,date,win,puzzleRating
0,nhQA7,2023-03-30 20:14:18.719,True,1877
1,0BoYj,2023-03-30 20:12:38.469,True,1643
2,Yq8Ld,2023-03-27 17:50:22.328,True,1962
3,49u5p,2023-03-27 17:49:40.924,True,1917
4,mQPeh,2023-03-27 17:49:04.291,True,1798


### Save it for later

We save the resulting data for future use.

In [ ]:
puzzle_activity_df.to_csv('tclark_puzzle_activity_clean.csv', index = False)

# My puzzle rating history
---

## Collection 

The puzzle history was retrieved directly from the Lichess API as a `.json` file. Simply go to

>https://lichess.org/api/user/{username}/rating-history

to access a user's rating history across all performance types (e.g. `Classical`, `Rapid`, `Blitz`, and `Puzzles`). 

## Processing

### Extracting puzzle performance

The `.json` file contains information on my rating history across *all* performance types—we'll need to extract the data from my puzzle rating history.

First, let's convert the file to a dataframe.

In [ ]:
rating_history_df = pd.read_json('https://lichess.org/api/user/tclark/rating-history')

rating_history_df

,name,points
0,Bullet,[]
1,Blitz,[]
2,Rapid,[]
3,Classical,[]
4,Correspondence,[]
5,Chess960,[]
6,King of the Hill,[]
7,Three-check,[]
8,Antichess,[]
9,Atomic,[]


In the `points` column is a list data, e.g.

>`[2022, 1, 20, 1423]`.

The first 3 entries represent year, month, and day. The last entry is my puzzle rating at the end of that day. Interestingly, the month is indexed with `January` corresponding to `0`.

Let's extract my puzzle rating history from `rating_history_df` into it's own dataframe.



In [ ]:
rating_list = rating_history_df.iloc[13,1]
puzzle_rating_history_df = pd.DataFrame(rating_list, columns = ['year', 'month', 'day', 'rating'])

puzzle_rating_history_df.head()

,year,month,day,rating
0,2022,1,20,1423
1,2022,2,12,1482
2,2022,2,13,1443
3,2022,2,14,1486
4,2022,2,15,1510


We can see that I started my puzzling journey with a humble rating of `1423`. I eventually improved, as shown in the last few rows of the `puzzle_rating_history_df`.

In [ ]:
puzzle_rating_history_df.tail()

,year,month,day,rating
303,2023,2,10,2464
304,2023,2,15,2420
305,2023,2,17,2435
306,2023,2,18,2420
307,2023,2,19,2424


My maximum puzzle rating of `2510`, achieved after about `1` year and `300` puzzles, which we can see below.

In [ ]:
max_rating = puzzle_rating_history_df[['rating']].max()

puzzle_rating_history_df[puzzle_rating_history_df['rating'] == int(max_rating)]

,year,month,day,rating
293,2023,1,24,2510


### Missing or null values

Let's check for any missing or null values, just in case.

In [ ]:
puzzle_rating_history_df.isna().sum(axis = 0)

year      0
month     0
day       0
rating    0
dtype: int64

There are no missing or null values!

### Fixing the `month` column

As noted above, the `month` column indexes the months with `January` corresponding to `0`. This seems a little unnatural to me, so we'll simply add `1` to each of the entries in `month`.


In [ ]:
new_months = puzzle_rating_history_df['month'].map(lambda x: x + 1)
puzzle_rating_history_df['month'] = new_months

puzzle_rating_history_df.head()

,year,month,day,rating
0,2022,2,20,1423
1,2022,3,12,1482
2,2022,3,13,1443
3,2022,3,14,1486
4,2022,3,15,1510


### Save it for later

We save the resulting data for future use.

In [ ]:
puzzle_rating_history_df.to_csv('tclark_puzzle_rating_history_clean.csv', index = False)